In [41]:
from dataclasses import dataclass
from functools import partial
from typing import Callable, Optional

import jax
import jax.numpy as jnp
from jax import jit, vmap, grad, random, lax

import os
import pwd
os.chdir("/home/zongchen/thinned_mfld")
from utils.configs import CFG
from utils.problems import Problem
from jaxtyping import Array 
from jax_tqdm import scan_tqdm


In [42]:
def q1_nn(z, x):
    # Simple 2-layer NN for demonstration
    d_hidden = x.shape[0] - 2
    W1, b1, W2 = x[:d_hidden], x[d_hidden+1], x[d_hidden+2]
    h = jnp.tanh(z @ W1 + b1)
    return jnp.dot(W2, h)

In [55]:

Z = jnp.array([[0.5, 1.0], [1.5, -0.5], [-1.0, 2.0], [0.0, 0.0]])
Y = jnp.array([1.0, -1.0, 0.5, 0.2])
d_data = Z.shape[1]
N_particle = 3
x_all = jax.random.normal(jax.random.PRNGKey(0), (N_particle, d_data+2))  # 3 samples of NN params

preds_all = jax.vmap(q1_nn, in_axes=(None, 0))(Z, x_all)
preds = preds_all.mean(axis=0)
print("Predictions:", preds)
print("Recaled Predictions:", preds * N_particle)

_vm_q1 = vmap(vmap(q1_nn, in_axes=(None, 0)), in_axes=(0, None))  # (N,d) -> (N,)
_vm_grad_q1 = vmap(vmap(grad(q1_nn, argnums=1), in_axes=(None, 0)), in_axes=(0, None))      # (N,d) -> (N,d)

loss = 0.5 * jnp.mean((preds * N_particle - Y) ** 2)
print("Loss:", loss)

term1_coeff = _vm_q1(Z, x_all).sum(1) - Y  # (n, )
term1_vector = _vm_grad_q1(Z, x_all)       # (n, N, d)
term1_mean = (term1_coeff[:, None, None] * term1_vector).mean(0)  # (N,d)

print("vector field", term1_mean)
# print("Recaled vector field", term1_mean * N_particle)

Predictions: [0.5570277  0.4093504  0.4733181  0.70359534]
Recaled Predictions: [1.6710831 1.2280512 1.4199543 2.110786 ]
Loss: 1.2389978
vector field [[-0.01516087  0.00372981  0.         -0.04864987]
 [ 0.1363469   0.06518608  0.          0.3524648 ]
 [-1.3076844   0.42673165  0.         -0.949113  ]]


In [53]:
_vm_q1(Z, x_all).sum(0)

Array([-2.1759725e-01,  3.9007217e-03,  6.6435709e+00], dtype=float32)

In [47]:
def two_layer_nn(x, z):
    """Forward pass: x ∈ R^{d_in}, output ∈ R^{d_out}"""
    W1, b1, W2 = x[:, :d_data], x[:, d_data+1], x[:, d_data+2]
    h = jnp.tanh(z @ W1.T + b1)
    return h @ W2

preds = two_layer_nn(
    x=x_all,
    z=Z,
)
print("Predictions (two_layer_nn):", preds)

def loss(x_all, z, y):
    preds = two_layer_nn(x_all, z)
    return 0.5 * jnp.mean((preds - y) ** 2)

gradient = jax.grad(loss, argnums=0)(x_all, Z, Y)

print("Loss (two_layer_nn):", loss(x_all, Z, Y))
print("Gradient (two_layer_nn):", gradient)

Predictions (two_layer_nn): [1.6710831 1.2280512 1.4199543 2.110786 ]
Loss (two_layer_nn): 1.2389978
Gradient (two_layer_nn): [[-0.01516087  0.00372981  0.         -0.04864987]
 [ 0.13634692  0.06518609  0.          0.3524648 ]
 [-1.3076843   0.42673162  0.         -0.949113  ]]
